LightGBM_Modeling (Modified Version)


2. 라이브러리 & 기본 설정

In [8]:
import os
import json
import gc
import re
from pathlib import Path

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# 경로 설정 (Linear Regression과 동일)
DATA_DIR = Path("../feature_datasets")
OUTPUT_DIR = Path("results_lightgbm")
RESULTS_DIR = OUTPUT_DIR / "results"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data Source: {DATA_DIR}")
print(f"Output Path: {OUTPUT_DIR}")

# 재현성용 seed
RANDOM_SEED = 42

Data Source: ../feature_datasets
Output Path: results_lightgbm


In [9]:
def load_data(file_path):
    """Parquet 파일 로드"""
    print(f"Loading {file_path.name}...")
    df = pd.read_parquet(file_path)
    
    # 날짜순 정렬
    if 'date_index' in df.columns:
        df = df.sort_values('date_index').reset_index(drop=True)
    
    # pub_date → Date 변환
    if 'pub_date' not in df.columns:
        raise ValueError(f"'pub_date' column not found in {file_path.name}")
    
    df['Date'] = pd.to_datetime(df['pub_date'], format='%Y_%m_%d')
    
    print(f"   Loaded {len(df)} rows, {len(df.columns)} columns")
    print(f"   Date range: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")
    
    return df

3. 파일 이름 파싱 함수
- 파일명에서 Dataset/Method/Type 정보를 추출해서 결과 CSV에 넣어주기.
    - `dataset_A.parquet` → Dataset = 'A', Method=None, Type=None
    - `dataset_D_headlines_orig.parquet` → Dataset='D', Method='headlines', Type='orig'

In [10]:
def parse_dataset_filename(fname: str):
    """
    feature_datasets 안의 파일명을 파싱해
    Dataset / Method / Type 정보를 반환한다.
    """
    name = fname.replace("dataset_", "").replace(".parquet", "")
    parts = name.split("_")

    # Case 1: dataset_A.parquet → ['A']
    if len(parts) == 1:
        return {
            "Dataset": parts[0],   # 'A'
            "Method": None,
            "Type": None,
        }

    # Case 2: dataset_B_headlines_orig.parquet → ['B', 'headlines', 'orig']
    if len(parts) == 3:
        Dataset, Method, Type = parts
        return {
            "Dataset": Dataset,
            "Method": Method,
            "Type": Type,
        }

    # 예상 밖 패턴이 나오면 None
    return {
        "Dataset": None,
        "Method": None,
        "Type": None,
    }

5. Feature 컬럼 자동 선택 함수

- 이 함수의 핵심 목적:
    - **data leakage를 일으키는 컬럼, 메타데이터**를 자동으로 제거
    - lag, fg_lag, PCA, embedding, person one-hot 등 **실제 feature만 선택**
- 규칙은 Dataset D의 전체 스키마를 기준으로 설계.

In [11]:
def extract_feature_columns(df: pd.DataFrame):
    """
    DataFrame에서 '모델에 넣을 feature 컬럼'만 선택해서 리스트로 반환한다.

    Drop 대상:
      - value (target)
      - 날짜 관련: date_str, Date, date_index, article_date, pub_date
      - 텍스트 메타데이터: person(문자열), article_id, person_id, idx,
                           headline, trailText, bodyText, webTitle, webUrl,
                           apiUrl, wordcount
      - fg_value (현재 시점 Fear-Greed 지표 → leakage)

    Keep 대상(명시적):
      - lag_*, fg_lag_*, fg_value? (← 여기서는 드랍), pca_*, emb_*, person_*

    그 외 숫자형 컬럼은 기본적으로 feature로 허용.
    """

    DROP_PATTERNS = [
        r"^value$",
        r"^date_str$", r"^date$", r"^date_index$",
        r"article_date", r"pub_date",
        r"article_id", r"person_id", r"idx$",
        r"headline", r"trailText", r"bodyText",
        r"webUrl", r"apiUrl", r"webTitle", r"wordcount",
        r"^person$",     # 문자열 person (이름)
        r"^fg_value$",   # 현재 시점 fear-greed 값 → leakage
    ]

    KEEP_PATTERNS = [
        r"^lag_\d+$",
        r"^fg_lag_\d+$",
        r"^pca_\d+$",
        r"^emb_\d+$",
        r"^person_\d+$",  # one-hot (person_1 ~ person_100)
    ]

    def match_any(col: str, patterns):
        return any(re.search(p, col) for p in patterns)

    feature_cols = []

    for col in df.columns:
        # 1) Drop 대상이면 무조건 제외
        if match_any(col, DROP_PATTERNS):
            continue

        # 2) Keep 패턴에 걸리면 무조건 포함
        if match_any(col, KEEP_PATTERNS):
            feature_cols.append(col)
            continue

        # 3) 위에 걸리지 않았는데 numeric type이면 feature로 사용
        if pd.api.types.is_numeric_dtype(df[col]):
            feature_cols.append(col)

    print(f"   >>> Selected {len(feature_cols)} feature columns.")
    return feature_cols

6. 전처리 + Train/Valid/Test Split + Sample Weight 계산
### 6.1 날짜 처리 & Split 기준
- `date_str` 형식: `"YYYY_MM_DD"`
- 이를 `datetime`으로 변환한 `date` 컬럼을 생성
- Split 기준:
    - Train: 2017-01-01 ~ 2018-12-31
    - Valid: 2019-01-01 ~ 2019-06-30
    - Test : 2019-07-01 ~ 2019-12-31
### 6.2 Sample Weight
- 같은 날짜에 기사 N개 → 각 행의 가중치 = 1 / N
- 이렇게 하면 하루 단위 loss contribution이 모두 비슷해짐.

In [12]:
def preprocess_and_split(df: pd.DataFrame):
    """
    1) feature 컬럼 자동 선택
    2) Train/Valid/Test Split
    3) 날짜별 기사 수 기반 sample weight 계산
    """

    # 1. Date 컬럼 확인 (이미 load_data에서 생성됨)
    if 'Date' not in df.columns:
        raise ValueError("'Date' column not found in dataframe")
    
    # date 컬럼 생성 (기존 코드와 호환성 유지)
    df["date"] = df["Date"]

    # 2. feature 컬럼 선택 (data leakage 방지 핵심)
    feature_cols = extract_feature_columns(df)

    # 3. Split 마스크 정의
    train_mask = (df["date"] >= "2017-01-01") & (df["date"] <= "2018-12-31")
    valid_mask = (df["date"] >= "2019-01-01") & (df["date"] <= "2019-06-30")
    test_mask  = (df["date"] >= "2019-07-01") & (df["date"] <= "2019-12-31")

    # 5. 날짜별 sample weight 계산 함수
    def make_X_y_w_dates(sub_df: pd.DataFrame):
        """
        subset DataFrame에 대해:
          - X: feature matrix
          - y: target (value)
          - w: sample weight (1 / 날짜별 기사 수)
          - d: 날짜 문자열 (JSON/평가 저장용)
        """
        if len(sub_df) == 0:
            return None, None, None, None

        if "value" not in sub_df.columns:
            raise ValueError("target 컬럼 'value'가 없습니다.")

        # 날짜 기준 기사 개수
        # date_index가 있으면 그걸 쓰고, 없으면 date를 사용
        if "date_index" in sub_df.columns:
            key = sub_df["date_index"]
        else:
            key = sub_df["date"]

        counts = key.value_counts()
        weights = 1.0 / key.map(counts)

        X = sub_df[feature_cols]
        y = sub_df["value"].astype(float)
        d = sub_df["date"].dt.strftime("%Y-%m-%d")   # JSON 저장용 포맷

        return X, y, weights, d

    # 실제 split
    train_df = df[train_mask].copy()
    valid_df = df[valid_mask].copy()
    test_df  = df[test_mask].copy()

    X_train, y_train, w_train, _       = make_X_y_w_dates(train_df)
    X_valid, y_valid, w_valid, _       = make_X_y_w_dates(valid_df)
    X_test,  y_test,  w_test, d_test   = make_X_y_w_dates(test_df)

    print(f"   >>> Split sizes | train: {len(train_df)}, valid: {len(valid_df)}, test: {len(test_df)}")
    return (X_train, y_train, w_train), (X_valid, y_valid, w_valid), (X_test, y_test, w_test, d_test)

7. LightGBM 학습/예측 프로세스 (파일 하나 기준)

- 이 함수는 **하나의 parquet 파일(dataset_*.parquet)** 을 입력으로 받아:
    1. 데이터 로드
    2. 전처리 + split + weight 계산
    3. LightGBM 학습
    4. 기사 단위 예측 → 날짜별 평균
    5. 일별 MSE 계산
    6. JSON / metrics 리스트에 결과 기록

In [13]:
def run_lightgbm_for_file(fname: str, metrics: list):
    """
    단일 feature dataset 파일에 대해 LightGBM 학습 및 평가를 수행하고,
    결과를 metrics 리스트에 추가한다.
    """
    info = parse_dataset_filename(fname)
    Dataset = info["Dataset"]
    Method  = info["Method"]
    Type    = info["Type"]

    print(f"\n>>> Processing file: {fname}")
    print(f"    Dataset={Dataset}, Method={Method}, Type={Type}")

    # 1. 데이터 로드 (load_data 함수 사용)
    file_path = DATA_DIR / fname
    df = load_data(file_path)

    # 2. 전처리 + split + sample weight
    train_set, valid_set, test_set = preprocess_and_split(df)
    X_train, y_train, w_train = train_set
    X_valid, y_valid, w_valid = valid_set
    X_test,  y_test,  w_test, d_test = test_set

    if X_train is None or X_valid is None or X_test is None:
        print("   [Warning] Empty split (train/valid/test 중 일부가 비어 있음). 스킵합니다.")
        return

    # 3. LightGBM Dataset 생성
    dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid, weight=w_valid, reference=dtrain)

    # 4. LightGBM 파라미터 (v4.x 호환)
    params = {
        "objective": "regression",
        "metric": "mse",
        "learning_rate": 0.05,
        "num_leaves": 31,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "seed": RANDOM_SEED,
        "verbosity": -1,
        "n_jobs": -1,
    }

    callbacks = [
        lgb.early_stopping(stopping_rounds=50, verbose=False),
        lgb.log_evaluation(period=100),
    ]

    # 5. 학습
    print("   >>> Training LightGBM ...")
    model = lgb.train(
        params=params,
        train_set=dtrain,
        num_boost_round=1000,
        valid_sets=[dtrain, dvalid],
        valid_names=["train", "valid"],
        callbacks=callbacks,
    )

    # 6. 기사 단위 예측
    print("   >>> Predicting on test set ...")
    y_pred_raw = model.predict(X_test, num_iteration=model.best_iteration)

    # 7. 날짜별 집계
    #    - 같은 날짜에 여러 기사가 있을 수 있으므로
    #      actual은 first(), pred는 mean()으로 집계
    res_df = pd.DataFrame({
        "date": d_test.values,
        "actual": y_test.values,
        "pred": y_pred_raw,
    })

    daily_actual = res_df.groupby("date")["actual"].first()
    daily_pred   = res_df.groupby("date")["pred"].mean()
    daily_df     = pd.concat([daily_actual, daily_pred], axis=1)

    # 8. 일별 MSE 계산
    mse = mean_squared_error(daily_df["actual"], daily_df["pred"])
    print(f"   [Result] Daily MSE: {mse:.6f}")

    # 9. metrics 리스트에 기록 (나중에 CSV로 저장)
    metrics.append({
        "Feature_set": Dataset,
        "Embeddings": Method if Method is not None else "-",
        "Dim_reduction": Type if Type is not None else "-",
        "Model": "LightGBM",
        "MSE": mse,
    })

    # 10. 일별 예측 결과 JSON 저장
    #     구조: { "YYYY-MM-DD": { "actual": ..., "pred": ... }, ... }
    json_obj = {
        date: {
            "actual": float(row["actual"]),
            "pred": float(row["pred"]),
        }
        for date, row in daily_df.iterrows()
    }

    # 데이터셋 이름 생성
    if Dataset == "A":
        dataset_name = "A"
    else:
        dataset_name = f"{Dataset}_{Method or 'none'}_{Type or 'none'}"
    
    json_name = f"pred_LightGBM_{dataset_name}.json"
    json_path = RESULTS_DIR / json_name
    
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_obj, f, indent=2)
    print(f"   >>> Saved prediction JSON to: {json_path}")

    # 11. 메모리 정리
    del df, dtrain, dvalid, model, X_train, X_valid, X_test
    gc.collect()

8. 전체 파일에 대해 LightGBM 실행
- feature_datasets 폴더 안의 모든 `dataset_*.parquet` 파일에 대해
  위에서 정의한 `run_lightgbm_for_file()`를 순차 실행
- 실행이 성공한 경우 metrics 리스트에 한 줄씩 기록
- 마지막에 metrics를 하나의 CSV로 저장:
    - Columns: Dataset / Method / Type / Model / MSE

In [14]:
def main():
    if not DATA_DIR.exists():
        print("[Error] feature_datasets 폴더를 찾을 수 없습니다.")
        return

    # 처리할 파일 목록
    file_list = sorted(
        [f for f in os.listdir(DATA_DIR) if f.startswith("dataset_") and f.endswith(".parquet")]
    )
    print(f"Found {len(file_list)} dataset files.")

    metrics = []

    for fname in file_list:
        try:
            run_lightgbm_for_file(fname, metrics)
        except Exception as e:
            print(f"   [Error] Failed processing {fname}: {e}")

    # 결과를 CSV로 저장
    if metrics:
        metrics_df = pd.DataFrame(metrics)
        metrics_df = metrics_df.sort_values(["Feature_set", "Embeddings", "Dim_reduction"])

        csv_path = OUTPUT_DIR / "lightgbm_evaluation_metrics.csv"
        metrics_df.to_csv(csv_path, index=False)
        print("\n[Done] All files processed.")
        print("Metrics saved to:", csv_path)
        print(metrics_df)
    else:
        print("No metrics were produced. (모든 파일이 에러로 스킵된 듯 합니다.)")

if __name__ == "__main__":
    main()

Found 25 dataset files.

>>> Processing file: dataset_A.parquet
    Dataset=A, Method=None, Type=None
Loading dataset_A.parquet...
   Loaded 754 rows, 9 columns
   Date range: 2017-01-03 to 2019-12-31
   >>> Selected 5 feature columns.
   >>> Split sizes | train: 502, valid: 124, test: 128
   >>> Training LightGBM ...
[100]	train's l2: 244.718	valid's l2: 479.234
[200]	train's l2: 166.844	valid's l2: 467.409
   >>> Predicting on test set ...
   [Result] Daily MSE: 21608.745003
   >>> Saved prediction JSON to: results_lightgbm/results/pred_LightGBM_A.json

>>> Processing file: dataset_B_bodyText_orig.parquet
    Dataset=B, Method=bodyText, Type=orig
Loading dataset_B_bodyText_orig.parquet...
   Loaded 460722 rows, 1037 columns
   Date range: 2017-01-01 to 2019-12-31
   >>> Selected 1029 feature columns.
   >>> Split sizes | train: 312185, valid: 76136, test: 72401
   >>> Training LightGBM ...
[100]	train's l2: 35.8323	valid's l2: 484.809
   >>> Predicting on test set ...
   [Result] Dai